# Using Aim with Tune

(tune-aim-ref)=

[Aim](https://aimstack.io) is an easy-to-use & supercharged open-source experiment tracker.
Aim logs your training runs, enables a beautiful UI to compare them and an API to query them programmatically.

```{image} /images/aim_logo_full.png
:align: center
:alt: Aim
:width: 100%
:target: https://aimstack.io
```

Ray Tune currently offers built-in integration with Aim.
The {ref}`AimCallback <tune-aim-logger>` allows for automatic logging of metrics reported to Tune to the Aim API.


```{contents}
:backlinks: none
:local: true
```

## Running An Aim Example

The following example demonstrates how the `AimCallback` works in a toy project.
We begin by importing the necessary modules:

In [ ]:
import numpy as np

import ray
from ray import air, tune
from ray.air import session
from ray.tune.logger.aim import AimCallback

Next, we'll define a simple `train_function` (a Tune `Trainable`) that reports a random loss to Tune.
The objective function itself is not important for this example, as our main focus is on the integration with Aim.

In [2]:
def train_function(config):
    for _ in range(50):
        loss = config["mean"] + config["sd"] * np.random.randn()
        session.report({"loss": loss})

Here is an example of how you can use the `AimCallback` to set up a simple grid-search Tune run.

In [3]:
def tune_with_callback():
     """An example of utilizing an AimCallback"""
     tuner = tune.Tuner(
         train_function,
         tune_config=tune.TuneConfig(
             metric="loss",
             mode="min",
         ),
         run_config=air.RunConfig(
             callbacks=[
                 AimCallback(repo="ray_tune", experiment_name="ray tune example", as_multirun=True)
             ]
         ),
         param_space={
             "mean": tune.grid_search([1, 2, 3, 4, 5, 6, 7, 8, 9]),
             "sd": tune.uniform(0.1, 0.9),
         },
     )
     tuner.fit()

In [ ]:
tune_with_callback()

Upon executing the script, a grid-search will be carried out and the results will be seamlessly saved to the Aim repo.
These results can then be easily accessed and analyzed through the user-friendly interface provided by Aim UI, stored in the `ray_tune` repository.

```{image} /images/aim_example_metrics_page.png
:align: center
:alt: Aim Metrics Explorer
:target: https://aimstack.readthedocs.io/en/latest/ui/pages/explorers.html#metrics-explorer
```

This concludes our examination of Tune with Aim.
In the sections to come, you'll find more in-depth information on the API of the Tune-Aim integration.

## Tune Aim API Reference

### AimCallback

(tune-aim-logger)=

```{eval-rst}
.. autoclass:: ray.tune.logger.aim.AimCallback
   :noindex:
```